# Images augmentation with torchvision.transforms

Deep learning models usually require a lot of data for training. In general, the more the data, the better the performance of the model. But acquiring massive amounts of data comes with its own challenges. Instead of spending days manually collecting data, we can make use of Image augmentation techniques. Image augmentation helps spruce up existing images without having to put manual time taking efforts.

Image Augmentation is the process of generating new images for the training CNN model. These new images are generated from the existing training images and hence we don’t have to do them manually.

In this notebook, we will implement  some of image augmentation methods using torchvision.transforms for the dataset of the competition: Cassava Leaf Disease Classification

# Loading the Libraries 

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import glob
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import numpy as np
import pandas as pd
import os

# Preparing the Dataset class

In [ ]:
class LeafDiseaseDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        return torch.tensor(img,dtype=torch.float)

In [ ]:
# The function used to see images with the selected transformations
def show_img(img):
  plt.figure(figsize=(40,38))
  npimg=img.numpy()
  plt.imshow(np.transpose(npimg,(1,2,0)))
  plt.show()

In [ ]:
images_path = "../input/cassava-leaf-disease-classification/train_images"
training_csv_file = "../input/cassava-leaf-disease-classification/train.csv"

# Visualizing some images without any transformation

In [ ]:
transform = transforms.Compose(
        [
            transforms.ToTensor(),
        ]
    )

In [ ]:
dataloader=DataLoader(LeafDiseaseDataset(images_path,training_csv_file, transform= transform),batch_size=8,shuffle=True)
data=iter(dataloader)
show_img(torchvision.utils.make_grid(data.next(), nrow=4))

# Adding some images augmentation techniques

1. **Rotation**

Image rotation helps our model to become more robust to the changes in the orientation of objects. The information of the image remains the same. 
Let’s see how we can rotate it. we will use the RandomRotation function of the torchvision.transforms to rotate the image.

In [ ]:
# Add rotation
rot_transform=transforms.Compose([
                              transforms.RandomRotation(45),  
                              transforms.ToTensor(),
                              ])

In [ ]:
dataloader=DataLoader(LeafDiseaseDataset(images_path,training_csv_file, transform= rot_transform),batch_size=8,shuffle=True)
data=iter(dataloader)
show_img(torchvision.utils.make_grid(data.next(), nrow=4))

2. **Random Cropping**

The differently cropped image is the most important aspect of image diversity. When your network is used by the real users, the object in the image can be a different position.

In [ ]:
crop_transform=transforms.Compose([
                              transforms.RandomCrop((240,240)),        
                              transforms.ToTensor(),
                              ])

In [ ]:
dataloader=DataLoader(LeafDiseaseDataset(images_path,training_csv_file, transform= crop_transform),batch_size=8,shuffle=True)
data=iter(dataloader)
show_img(torchvision.utils.make_grid(data.next(), nrow=4))

3. **Flipping images**

Your network will be trained on batches of images which are randomly flipped from the original dataset, and which are sometimes flipped probability = 0.5.
Flipping the images make more generalized models that will learn the patterns of the original as well as the flipped images. 

In [ ]:
flip_transform=transforms.Compose([
                              transforms.RandomVerticalFlip(0.5), 
                              transforms.RandomHorizontalFlip(0.5),        
                              transforms.ToTensor(),
                              ])

In [ ]:
dataloader=DataLoader(LeafDiseaseDataset(images_path,training_csv_file, transform= flip_transform),batch_size=8,shuffle=True)
data=iter(dataloader)
show_img(torchvision.utils.make_grid(data.next(), nrow=4))

4. **Brightness, Contrast, Saturation, Hue**

The quality of the images will not be the same from each source. Some images might be of very high quality while others might be just plain bad. In such scenarios, we can blur the image. This helps make our deep learning model more robust. Transforms provide a class for randomly change the brightness, contrast, and saturation of an image.

In [ ]:
specific_transform=transforms.Compose([
                              transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0, hue=0),
                              transforms.ToTensor(),
                              ])

In [ ]:
dataloader=DataLoader(LeafDiseaseDataset(images_path,training_csv_file, transform= specific_transform),batch_size=8,shuffle=True)
data=iter(dataloader)
show_img(torchvision.utils.make_grid(data.next(), nrow=4))

5. **Gaussian Noise to Images**

Adding random noise to the images is also an image augmentation technique. It can be done with a Gaussian filter for blurring the image. 

In [ ]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

Transforms give us fine-grained control of the transformation pipeline. we can use a functional transform to build transform classes with custom behavior.

In [ ]:
gauss_transform=transforms.Compose([
                              transforms.ToTensor(),
                              AddGaussianNoise(0.1, 0.08)
                              ])

In [ ]:
dataloader=DataLoader(LeafDiseaseDataset(images_path,training_csv_file, transform= gauss_transform),batch_size=8,shuffle=True)
data=iter(dataloader)
show_img(torchvision.utils.make_grid(data.next(), nrow=4))

6. **Random Erasing**

Randomly selects a rectangle region in an image and erases its pixels with random values. In this process, training images with various levels of occlusion are generated, which reduces the risk of over-fitting and makes the model robust to occlusion.

In [ ]:
erase_transform=transforms.Compose([  
                              transforms.ToTensor(),
                              transforms.RandomErasing(),  
                              ])

In [ ]:
dataloader=DataLoader(LeafDiseaseDataset(images_path,training_csv_file, transform= erase_transform),batch_size=8,shuffle=True)
data=iter(dataloader)
show_img(torchvision.utils.make_grid(data.next(), nrow=4))

# Conclusion

These are many image augmentation techniques which help to make our deep learning model robust and generalizable. This also helps increase the size of the training set for small datasets.

All transformations somehow change the image. They leave the original untouched, just returning a changed copy. Given the same input image, some methods will always apply the same changes(e.g., converting it to Tensor, resizing to a fixed shape, etc.). Other methods will apply transformations with random parameters, returning different results each time (e.g., randomly cropping the images, randomly changing their brightness or saturation, etc.).So that means that upon every epoch you get a different version of the dataset, You can apply many data augmentation techniques together via  transforms.Compose().

The purpose of data augmentation is to try to get an upper bound of the data distribution of unseen data.
here's the link for the resources used : 
* https://towardsdatascience.com/image-augmentation-mastering-15-techniques-and-useful-functions-with-python-codes-44c3f8c1ea1f
* https://androidkt.com/pytorch-image-augmentation-using-transforms/#:~:text=techniques%20using%20torchvision.-,transforms.,have%20to%20do%20them%20manually.